In [4]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import cantera as ct

from Project_3_3 import solve_pfr_cantera, solve_pfr_pyrolysis


# ---------------------------------------------------------------------------
# NumPy-only cumulative trapezoidal integrator
# ---------------------------------------------------------------------------
def cumtrapz_np(y, x):
    out = np.zeros_like(y)
    for i in range(1, len(y)):
        out[i] = out[i-1] + 0.5 * (y[i] + y[i-1]) * (x[i] - x[i-1])
    return out


# ---------------------------------------------------------------------------
# Compute residence time from PFR solution (no SciPy)
# ---------------------------------------------------------------------------
def compute_residence_time(x, T, Y, p0, mechfile, mdot, D):
    gas = ct.Solution(mechfile)
    A = np.pi * D**2 / 4.0
    rho = np.zeros_like(x)
    u   = np.zeros_like(x)

    for j in range(len(x)):
        gas.TPY = T[j], p0, Y[:, j]
        rho[j] = gas.density
        u[j]   = mdot / (rho[j] * A)

    tau = cumtrapz_np(1/u, x)
    return tau


# =============================================================================
# Verification 4.1
# =============================================================================
def run_verification_41():
    print("\n=== Running Verification 4.1 ===")

    mech = "gri30.yaml"
    T0 = 300.0
    Tw = 1200.0
    p0 = ct.one_atm
    mdot = 0.1
    D = 0.05
    x_end = 1.0

    h_val = 50.0
    cp_val = 1000.0

    def h_const(T, Y, p, rho, u): return h_val
    def cp_const(T): return cp_val

    sol = solve_pfr_cantera(
        mechfile=mech,
        X0="N2:1",
        T0=T0,
        p0=p0,
        mdot=mdot,
        D=D,
        Tw=Tw,
        x_end=x_end,
        Npts=200,
        h_override=h_const,
        cp_override=cp_const
    )

    x = sol["x"]
    T_num = sol["T"]

    Per = np.pi * D
    K = h_val * Per / (mdot * cp_val)
    T_anal = Tw + (T0 - Tw)*np.exp(-K*x)

    # Plot
    plt.figure(figsize=(7,5))
    plt.plot(x, T_num, label="Numerical")
    plt.plot(x, T_anal, '--', label="Analytic")
    plt.legend(); plt.grid(True)
    plt.xlabel("x (m)"); plt.ylabel("T (K)")
    plt.title("Verification 4.1: Constant-h Analytic Check")
    plt.tight_layout()
    plt.savefig("verif_41.png")
    plt.close()

    print("✓ Verification 4.1 complete (saved: verif_41.png)")


# =============================================================================
# Verification 4.2 and 4.3 (h=0 vs constant-pressure reactor)
# =============================================================================
def run_verification_42_43(label, mech, X0, T0):
    print(f"\n=== Running Verification {label} ===")

    p0 = ct.one_atm
    mdot = 0.1
    D = 0.05
    x_end = 0.5

    # h = 0 => adiabatic
    def h_zero(T, Y, p, rho, u): return 0.0

    sol = solve_pfr_cantera(
        mechfile=mech,
        X0=X0,
        T0=T0,
        p0=p0,
        mdot=mdot,
        D=D,
        Tw=T0,            # irrelevant since h=0
        x_end=x_end,
        Npts=300,
        h_override=h_zero
    )

    x = sol["x"]
    T_pfr = sol["T"]
    Y_pfr = sol["Y"]
    mdot_pfr = sol["mdot"]

    tau = compute_residence_time(x, T_pfr, Y_pfr, p0, mech, mdot_pfr, D)

    # Cantera constant-pressure batch reactor
    gas = ct.Solution(mech)
    gas.TPX = T0, p0, X0
    r = ct.IdealGasConstPressureReactor(gas)
    sim = ct.ReactorNet([r])

    T_batch = np.zeros_like(tau)
    for j, t in enumerate(tau):
        sim.advance(t)
        T_batch[j] = r.T

    plt.figure(figsize=(7,5))
    plt.plot(tau, T_pfr, label="PFR (h=0)")
    plt.plot(tau, T_batch, '--', label="Constant-P Reactor")
    plt.xlabel("τ (s)"); plt.ylabel("T (K)")
    plt.title(f"Verification {label}: Temperature Comparison")
    plt.grid(True); plt.legend(); plt.tight_layout()
    plt.savefig(f"verif_{label}.png")
    plt.close()

    print(f"✓ Verification {label} complete (saved: verif_{label}.png)")


def run_verification_4():
    run_verification_41()
    run_verification_42_43("4.2", "gri30.yaml", "H2:2, O2:1, N2:4", 1200.0)
    run_verification_42_43("4.3", "gri30.yaml", "CH4:1, O2:2, N2:7.52", 1100.0)


# =============================================================================
# Problem 5: CH4 Pyrolysis for Four Mechanisms
# =============================================================================
def run_problem5():
    print("\n=== Running Problem 5: CH4 Pyrolysis ===")

    mechanisms = {
        "GRI-Mech 3.0": "gri30.yaml",
        "USC Mech II":  "USCII.yaml",
        "ABF Soot":     "abf.yaml",
        "Cai-Pitsch":   "cai_pisch.yaml"
    }

    # PFR operating conditions
    D = 0.05
    u0 = 0.05
    T0 = 1200.0
    Tw = 2000.0
    p0 = ct.one_atm
    x_end = 20.0

    results = {}

    for name, mech in mechanisms.items():
        print(f"  → Running {name} ...")
        try:
            sol = solve_pfr_pyrolysis(
                mechfile = mech,
                X0 = {"CH4":1.0},
                T0 = T0,
                p0 = p0,
                u0 = u0,
                D = D,
                Tw = Tw,
                x_end = x_end,
                Npts = 400
            )
            results[name] = sol
            print("    ✓ Success")
        except Exception as e:
            print(f"    ✗ Failed: {e}")

    # Plot CH4
    plt.figure(figsize=(7,5))
    for name, sol in results.items():
        gas = sol["gas"]; X = sol["X"]
        idx = gas.species_index("CH4")
        plt.plot(sol["x"], X[idx], label=name)
    plt.xlabel("x (m)"); plt.ylabel("CH4 mole fraction")
    plt.title("Problem 5: CH4 Mole Fraction Comparison")
    plt.grid(True); plt.legend(); plt.tight_layout()
    plt.savefig("prob5_CH4.png"); plt.close()

    # Plot H2
    plt.figure(figsize=(7,5))
    for name, sol in results.items():
        gas = sol["gas"]; X = sol["X"]
        idx = gas.species_index("H2")
        plt.plot(sol["x"], X[idx], label=name)
    plt.xlabel("x (m)"); plt.ylabel("H2 mole fraction")
    plt.title("Problem 5: H2 Mole Fraction Comparison")
    plt.grid(True); plt.legend(); plt.tight_layout()
    plt.savefig("prob5_H2.png"); plt.close()

    # Plot C2H2
    plt.figure(figsize=(7,5))
    for name, sol in results.items():
        gas = sol["gas"]; X = sol["X"]

        if "C2H2" in gas.species_names:
            idx = gas.species_index("C2H2")
            plt.plot(sol["x"], X[idx], label=name)
        else:
            print(f"    ⚠ {name} mechanism does not contain C2H2")

    plt.xlabel("x (m)"); plt.ylabel("C2H2 mole fraction")
    plt.title("Problem 5: C2H2 Mole Fraction Comparison")
    plt.grid(True); plt.legend(); plt.tight_layout()
    plt.savefig("prob5_C2H2.png"); plt.close()

    # Plot C2H4
     plt.figure(figsize=(7,5))
    for name, sol in results.items():
        gas = sol["gas"]; X = sol["X"]

        if "C2H4" in gas.species_names:
            idx = gas.species_index("C2H4")
            plt.plot(sol["x"], X[idx], label=name)
        else:
            print(f"    ⚠ {name} mechanism does not contain C2H4")

    plt.xlabel("x (m)"); plt.ylabel("C2H4 mole fraction")
    plt.title("Problem 5: C2H4 Mole Fraction Comparison")
    plt.grid(True); plt.legend(); plt.tight_layout()
    plt.savefig("prob5_C2H4.png"); plt.close()

    # Temperature
    plt.figure(figsize=(7,5))
    for name, sol in results.items():
        plt.plot(sol["x"], sol["T"], label=name)
    plt.axhline(Tw, linestyle='--', color='k', label="Wall T")
    plt.xlabel("x (m)"); plt.ylabel("Temperature (K)")
    plt.title("Problem 5: Temperature Comparison")
    plt.grid(True); plt.legend(); plt.tight_layout()
    plt.savefig("prob5_T.png"); plt.close()

    print("✓ Problem 5 complete (plots saved)")


# =============================================================================
# MAIN
# =============================================================================
def main():
    run_verification_4()
    run_problem5()
    print("\nAll project outputs generated!")


if __name__ == "__main__":
    main()


=== Running Verification 4.1 ===
✓ Verification 4.1 complete (saved: verif_41.png)

=== Running Verification 4.2 ===
✓ Verification 4.2 complete (saved: verif_4.2.png)

=== Running Verification 4.3 ===
✓ Verification 4.3 complete (saved: verif_4.3.png)

=== Running Problem 5: CH4 Pyrolysis ===
  → Running GRI-Mech 3.0 ...
    ✓ Success
  → Running USC Mech II ...
    ✓ Success
  → Running ABF Soot ...
    ✓ Success
  → Running Cai-Pitsch ...
    ✓ Success
✓ Problem 5 complete (plots saved)

All project outputs generated!
